In [ ]:
#!pip install pytorch-lightning --quiet
#!pip install transformers --quiet
#!pip install accelerate --quiet
#!pip install pytorch-forecasting --quiet

In [ ]:
# Core data & ML libs
# !pip install numpy --quiet
# !pip install pandas --quiet
# !pip install scikit-learn --quiet
# !pip install torch --quiet
# !pip install matplotlib --quiet
# !pip install seaborn --quiet
# !pip install scipy --quiet
# !pip install boto3 --quiet
# !pip install requests --quiet

# Time-series / forecasting
# !pip install pytorch-lightning --quiet
# !pip install pytorch-forecasting --quiet

# (Optional, if you need them)
# !pip install transformers --quiet
# !pip install accelerate --quiet

In [ ]:
# === minimal imports actually used in this notebook ===
import os
import requests
import numpy as np
import pandas as pd
from collections import deque

from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error

In [ ]:
# === download EirGrid XLSX reports from the provided links ===
DOWNLOAD_DIR = r"D:\Colab\Summer Project\Eirgrid\Reports\downloaded"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

urls = {
    "System-Data-Qtr-Hourly-2014-2015.xlsx": "https://www.eirgrid.ie/site-files/library/EirGrid/System-Data-Qtr-Hourly-2014-2015.xlsx",
    "System-Data-Qtr-Hourly-2016-2017.xlsx": "https://www.eirgrid.ie/site-files/library/EirGrid/System-Data-Qtr-Hourly-2016-2017.xlsx",
    "System-Data-Qtr-Hourly-2018-2019.xlsx": "https://www.eirgrid.ie/site-files/library/EirGrid/System-Data-Qtr-Hourly-2018-2019.xlsx",
    "System-Data-Qtr-Hourly-2020-2021.xlsx": "https://www.eirgrid.ie/site-files/library/EirGrid/System-Data-Qtr-Hourly-2020-2021.xlsx",
    "System-Data-Qtr-Hourly-2022-2023_0.xlsx": "https://www.eirgrid.ie/site-files/library/EirGrid/System-Data-Qtr-Hourly-2022-2023_0.xlsx",
    "System-Data-Qtr-Hourly-2024.xlsx": "https://www.eirgrid.ie/site-files/library/EirGrid/System-Data-Qtr-Hourly-2024.xlsx",
    "System-Data-Qtr-Hourly-2025.xlsx": "https://www.eirgrid.ie/site-files/library/EirGrid/System-Data-Qtr-Hourly-2025.xlsx",
}

downloaded_paths = {}
for fname, url in urls.items():
    out_path = os.path.join(DOWNLOAD_DIR, fname)
    if not os.path.exists(out_path):
        r = requests.get(url, timeout=120)
        r.raise_for_status()
        with open(out_path, "wb") as f:
            f.write(r.content)
    downloaded_paths[fname] = out_path

# convenience aliases for the rest of the notebook (match prior variable names)
file_2014_2015 = downloaded_paths["System-Data-Qtr-Hourly-2014-2015.xlsx"]
file_2016_2017 = downloaded_paths["System-Data-Qtr-Hourly-2016-2017.xlsx"]
file_2018_2019 = downloaded_paths["System-Data-Qtr-Hourly-2018-2019.xlsx"]
file_2020_2021 = downloaded_paths["System-Data-Qtr-Hourly-2020-2021.xlsx"]
file_2022_2023 = downloaded_paths["System-Data-Qtr-Hourly-2022-2023_0.xlsx"]
file_2024      = downloaded_paths["System-Data-Qtr-Hourly-2024.xlsx"]
file_2025      = downloaded_paths["System-Data-Qtr-Hourly-2025.xlsx"]

In [ ]:
# ===download ONLY interconnection data (2014 → today) ===
import requests
import pandas as pd

def fetch_interconnection(region: str = "ALL") -> pd.DataFrame:
    """
    Download monthly JSON slices from Smart Grid Dashboard and return a single
    DataFrame with columns: EffectiveTime (datetime), Value (float), Region (str).
    """
    # month starts from 2014-01 to the first day of next month (inclusive bound for loop)
    months = pd.period_range(start="2014-01", end=pd.Timestamp.today().strftime("%Y-%m"), freq="M")
    frames = []

    for per in months:
        start = per.start_time.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
        # next month "01-<mon>-YY 21:59" per reference implementation
        end   = (per.asfreq("M").to_timestamp() + pd.offsets.MonthBegin(1)).replace(hour=21, minute=59)

        # format like 01-Jan-14 HH:MM
        df_str = start.strftime("%d-%b-%y %H:%M")
        dt_str = end.strftime("%d-%b-%y %H:%M")

        url = (
            "https://www.smartgriddashboard.com/DashboardService.svc/data"
            f"?area=interconnection&region={region}"
            f"&datefrom={df_str.replace(':','%3A')}"
            f"&dateto={dt_str.replace(':','%3A')}"
        )

        try:
            r = requests.get(url, timeout=30)
            r.raise_for_status()
            rows = r.json().get("Rows", [])
            if not rows:
                continue
            frames.append(pd.DataFrame(rows))
        except Exception as e:
            print(f"⚠️ {region} {per.strftime('%Y-%m')}: {e}")

    if not frames:
        raise RuntimeError("No interconnection data retrieved.")

    df = pd.concat(frames, ignore_index=True)

    # Normalize expected columns
    # Keep only EffectiveTime + Value (+ Region if present)
    col_time = "EffectiveTime" if "EffectiveTime" in df.columns else "DateTime"
    col_val  = "Value" if "Value" in df.columns else "value"

    df = df[[c for c in [col_time, col_val, "Region"] if c in df.columns]].copy()
    df.rename(columns={col_time: "EffectiveTime", col_val: "Value"}, inplace=True)
    df["EffectiveTime"] = pd.to_datetime(df["EffectiveTime"], errors="coerce")
    df = df.dropna(subset=["EffectiveTime"]).sort_values("EffectiveTime").drop_duplicates("EffectiveTime")

    # If Region not supplied by API, stamp it
    if "Region" not in df.columns:
        df["Region"] = region

    return df

# Download ALL-region interconnection once; keep in memory
df_intercon_all = fetch_interconnection(region="ALL")

In [ ]:
# === Processing using the downloaded interconnection ===
# Columns needed across years
cols_df3 = [
    'DateTime',
    'NI Generation','NI Demand','NI Wind Availability','NI Wind Generation',
    'IE Generation','IE Demand','IE Wind Availability','IE Wind Generation',
    'SNSP'
]

def read_sysdata(path):
    return pd.read_excel(path, sheet_name="System Data", parse_dates=["DateTime"])

# Load per-period (files defined in part A after download)
df_2014_2015 = read_sysdata(file_2014_2015)[cols_df3]
df_2016_2017 = read_sysdata(file_2016_2017)[cols_df3]
df_2018_2019 = read_sysdata(file_2018_2019)[cols_df3]
df_2020_2021 = read_sysdata(file_2020_2021)[cols_df3]
df_2022_2023 = read_sysdata(file_2022_2023)[cols_df3]
df_2024      = read_sysdata(file_2024)[cols_df3]
df_2025      = read_sysdata(file_2025)[cols_df3]

# Stack all years
df_eir3 = pd.concat([
    df_2014_2015, df_2016_2017, df_2018_2019,
    df_2020_2021, df_2022_2023, df_2024, df_2025
], ignore_index=True)

# Enforce datetime and sort once
df_eir3['DateTime'] = pd.to_datetime(df_eir3['DateTime'])
df_eir3 = df_eir3.sort_values('DateTime').reset_index(drop=True)

# Use the freshly downloaded interconnection: take Value as the series
df_intercon = (
    df_intercon_all[['EffectiveTime', 'Value']]
    .rename(columns={'EffectiveTime': 'DateTime', 'Value': 'interconnection'})
)
df_intercon['DateTime'] = pd.to_datetime(df_intercon['DateTime'])

# Merge (left) without filling interconnection yet
df_eir3 = df_eir3.merge(df_intercon, on='DateTime', how='left')

# Quick check: sorted ascending
assert df_eir3['DateTime'].is_monotonic_increasing, "DateTime not sorted ascending."
print("Loaded & merged with downloaded interconnection. Rows:", len(df_eir3))


In [ ]:
# Fill non-target features to remove accidental gaps; keep interconnection untouched
_inter_orig = df_eir3['interconnection'].copy()
to_fill = [c for c in df_eir3.columns if c != 'interconnection']
df_eir3[to_fill] = df_eir3[to_fill].bfill().ffill()
df_eir3['interconnection'] = _inter_orig

# Set index for rolling ops
df_eir3 = df_eir3.set_index('DateTime')

# Helper: rolling means on 15-min data
def roll_mean(series, hours):
    window = int(hours * 4)  # 4 samples/hour
    return series.rolling(window=window, min_periods=window).mean()

# --- Non-interconnection features used in raw_top20 ---
# Net loads
df_eir3['NetLoad_IE'] = df_eir3['IE Demand'] - df_eir3['IE Wind Generation']
df_eir3['NetLoad_NI'] = df_eir3['NI Demand'] - df_eir3['NI Wind Generation']

# Rolling means required (1h = 4, 2h = 8, 4h = 16)
df_eir3['IE_WindGen_rollmean_1h'] = roll_mean(df_eir3['IE Wind Generation'], 1)
df_eir3['IE_WindGen_rollmean_2h'] = roll_mean(df_eir3['IE Wind Generation'], 2)
df_eir3['IE_WindGen_rollmean_4h'] = roll_mean(df_eir3['IE Wind Generation'], 4)

df_eir3['IE_Demand_rollmean_1h']  = roll_mean(df_eir3['IE Demand'], 1)
df_eir3['IE_Demand_rollmean_4h']  = roll_mean(df_eir3['IE Demand'], 4)

df_eir3['NI_Demand_rollmean_1h']  = roll_mean(df_eir3['NI Demand'], 1)
df_eir3['NI_Demand_rollmean_2h']  = roll_mean(df_eir3['NI Demand'], 2)
df_eir3['NI_Demand_rollmean_4h']  = roll_mean(df_eir3['NI Demand'], 4)

df_eir3['NI_WindGen_rollmean_1h'] = roll_mean(df_eir3['NI Wind Generation'], 1)

# --- Interconnection-based features (for training rows where interconnection exists) ---
# Lags
df_eir3['intercon_lag_1'] = df_eir3['interconnection'].shift(1)
df_eir3['intercon_lag_4'] = df_eir3['interconnection'].shift(4)

# 4h rolling mean (16 samples)
df_eir3['intercon_rollmean_4h'] = df_eir3['interconnection'].rolling(window=16, min_periods=16).mean()

# 4h slope via simple OLS slope on last 16 points
def rolling_slope(series, window=16):
    # compute slope for each rolling window using polyfit on index 0..window-1
    idx = np.arange(window)
    def _s(x):
        if np.isnan(x).any():
            return np.nan
        # slope only
        return np.polyfit(idx, x, 1)[0]
    return series.rolling(window=window, min_periods=window).apply(_s, raw=True)

df_eir3['intercon_slope_4h'] = rolling_slope(df_eir3['interconnection'], window=16)

# Restore DateTime as column
df_eir3 = df_eir3.reset_index()

# Minimal NaN report (mostly first ~16 rows due to roll windows or missing interconnection)
nan_counts = df_eir3.isna().sum()
print("NaNs present in:\n", nan_counts[nan_counts>0].sort_values(ascending=False).head(15))

In [ ]:
# The fixed top-20 features you derived
raw_top20 = [
    'intercon_lag_1',
    'IE_WindGen_rollmean_1h',
    'NetLoad_IE',
    'intercon_rollmean_4h',
    'IE Wind Generation',
    'IE_Demand_rollmean_4h',
    'NI_Demand_rollmean_2h',
    'IE Demand',
    'intercon_lag_4',
    'IE_Demand_rollmean_1h',
    'IE_WindGen_rollmean_4h',
    'NetLoad_NI',
    'NI Demand',
    'intercon_slope_4h',
    'NI_Demand_rollmean_1h',
    'IE Generation',
    'NI_WindGen_rollmean_1h',
    'NI_Demand_rollmean_4h',
    'IE_WindGen_rollmean_2h',
    'NI Wind Generation'
]

# Build training set: rows with real interconnection and all needed features available (after 16-sample warmup)
feat_present_mask = df_eir3[raw_top20].notna().all(axis=1)
train_mask = df_eir3['interconnection'].notna() & feat_present_mask

df_train = df_eir3.loc[train_mask].copy()

X_train = df_train[raw_top20]
y_train = df_train['interconnection']

# Scale + grid search (time-series CV)
scaler = StandardScaler()
X_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    columns=raw_top20,
    index=X_train.index
)

tscv = TimeSeriesSplit(n_splits=5)
rmse_scorer = make_scorer(
    lambda y_true, y_pred: -np.sqrt(mean_squared_error(y_true, y_pred)),
    greater_is_better=True
)

ridge_param_grid = {
    'alpha':    [1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100],
    'solver':   ['saga', 'sag'],
    'tol':      [1e-2, 1e-3, 1e-4],
    'max_iter': [500, 1000, 5000]
}

ridge_grid = GridSearchCV(
    estimator=Ridge(random_state=0),
    param_grid=ridge_param_grid,
    cv=tscv,
    scoring=rmse_scorer,
    n_jobs=-1,
    return_train_score=True
)
ridge_grid.fit(X_scaled, y_train)

best_params = ridge_grid.best_params_
final_ridge = Ridge(
    alpha=best_params['alpha'],
    solver=best_params['solver'],
    tol=best_params['tol'],
    max_iter=best_params['max_iter'],
    random_state=0
)
final_ridge.fit(X_scaled, y_train)

print("Best Ridge params:", best_params)
print("TS CV RMSE (neg scorer):", -ridge_grid.best_score_)


In [ ]:
# quick holdout using 2022-01-01 forward (requires those dates exist with real interconnection)
cutoff_val = pd.Timestamp("2022-01-01")
mask_val = (df_train['DateTime'] >= cutoff_val)
X_val = df_train.loc[mask_val, raw_top20]
y_val = df_train.loc[mask_val, 'interconnection']

X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=raw_top20, index=X_val.index)
y_pred_val = final_ridge.predict(X_val_scaled)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"Hold-out RMSE (>=2022-01-01): {rmse_val:.4f}")


In [ ]:
# Impute strictly after the date you chose; adjust if needed to match your earlier cut
start_impute = pd.Timestamp("2024-02-20 00:00:00")  # first timestamp to impute (exclusive of 2024-02-19)
end_impute   = pd.Timestamp("2025-04-30 23:45:00")  # inclusive end bound

# Build index to impute (ensure it exists in df_eir3)
mask_imp = (df_eir3['DateTime'] >= start_impute) & (df_eir3['DateTime'] <= end_impute)
imp_index = df_eir3.index[mask_imp]

if len(imp_index) == 0:
    print("No rows to impute in the requested range.")
else:
    # Initialize a 16-sample buffer with the last known interconnection values BEFORE start_impute
    pre_window_end = df_eir3.index[df_eir3['DateTime'] < start_impute]
    if len(pre_window_end) == 0:
        raise ValueError("No history before start_impute to seed the interconnection buffer.")

    # Gather last 16 known/pred interconnection values (must be non-NaN)
    # If your history still has NaNs close to the boundary, you may need to walk back further.
    hist_mask = (df_eir3['DateTime'] < start_impute) & (df_eir3['interconnection'].notna())
    last16 = df_eir3.loc[hist_mask, 'interconnection'].tail(16).values

    if len(last16) < 16:
        raise ValueError("Need at least 16 historical interconnection values to start imputation.")

    buffer = deque(last16.tolist(), maxlen=16)

    # Precompute scaled non-interconnection features for the whole dataset so we can grab them quickly
    # We'll reuse the same StandardScaler fitted on training, so at predict-time we build a row in raw_top20 order.
    # Non-interconnection predictors used in raw_top20 (already computed in Cell 2) are:
    # IE_WindGen_rollmean_1h/2h/4h, IE_Demand_rollmean_1h/4h, NI_Demand_rollmean_1h/2h/4h, NI_WindGen_rollmean_1h,
    # NetLoad_IE, NetLoad_NI, IE/NI Demand, IE/NI Wind Generation, IE Generation.

    for idx in imp_index:
        row = df_eir3.loc[idx]

        # Build the 4 interconnection features from the buffer
        intercon_lag_1 = buffer[-1]
        intercon_lag_4 = buffer[-4] if len(buffer) >= 4 else np.nan
        intercon_rollmean_4h = float(np.mean(buffer))
        # slope over last 16 points
        x = np.arange(len(buffer))
        slope = float(np.polyfit(x, np.array(buffer, dtype=float), 1)[0])

        # Collect the rest of the features directly from precomputed columns at this timestamp
        feat_values = {
            'intercon_lag_1': intercon_lag_1,
            'IE_WindGen_rollmean_1h': row['IE_WindGen_rollmean_1h'],
            'NetLoad_IE': row['NetLoad_IE'],
            'intercon_rollmean_4h': intercon_rollmean_4h,
            'IE Wind Generation': row['IE Wind Generation'],
            'IE_Demand_rollmean_4h': row['IE_Demand_rollmean_4h'],
            'NI_Demand_rollmean_2h': row['NI_Demand_rollmean_2h'],
            'IE Demand': row['IE Demand'],
            'intercon_lag_4': intercon_lag_4,
            'IE_Demand_rollmean_1h': row['IE_Demand_rollmean_1h'],
            'IE_WindGen_rollmean_4h': row['IE_WindGen_rollmean_4h'],
            'NetLoad_NI': row['NetLoad_NI'],
            'NI Demand': row['NI Demand'],
            'intercon_slope_4h': slope,
            'NI_Demand_rollmean_1h': row['NI_Demand_rollmean_1h'],
            'IE Generation': row['IE Generation'],
            'NI_WindGen_rollmean_1h': row['NI_WindGen_rollmean_1h'],
            'NI_Demand_rollmean_4h': row['NI_Demand_rollmean_4h'],
            'IE_WindGen_rollmean_2h': row['IE_WindGen_rollmean_2h'],
            'NI Wind Generation': row['NI Wind Generation'],
        }

        # Make a 1-row DataFrame in the correct column order
        X_one = pd.DataFrame([[feat_values[k] for k in raw_top20]], columns=raw_top20, index=[idx])

        # Scale and predict
        X_one_scaled = pd.DataFrame(scaler.transform(X_one), columns=raw_top20, index=[idx])
        y_hat = float(final_ridge.predict(X_one_scaled)[0])

        # Write back prediction and update buffer
        df_eir3.at[idx, 'interconnection'] = y_hat
        buffer.append(y_hat)

    print(f"Imputed interconnection for {len(imp_index)} timestamps from {start_impute} to {end_impute}.")


In [ ]:
# Keep only the 7 required columns
df_eir = df_eir3[['DateTime',
                  'IE Generation','IE Demand',
                  'IE Wind Availability','IE Wind Generation',
                  'SNSP','interconnection']].copy()

# Enforce time bounds
start_bound = pd.Timestamp("2014-01-01 00:00:00")
end_bound   = pd.Timestamp("2025-04-30 23:45:00")
df_eir = df_eir[(df_eir['DateTime'] >= start_bound) & (df_eir['DateTime'] <= end_bound)].copy()

# Sort and basic integrity checks
df_eir = df_eir.sort_values('DateTime').reset_index(drop=True)

# 15-min regularity assertion (no reindexing – just check)
dt = df_eir['DateTime'].diff().dropna().unique()
ok_15m = (len(dt) == 1) and (pd.Timedelta(minutes=15) == dt[0])
print("15-min regularity:", ok_15m)

# No missing values in the final 7 columns
missing_final = df_eir.isna().sum()
print("Final NaNs by column:\n", missing_final)

print("Final df_eir shape:", df_eir.shape)
df_eir.head()
